In [1]:
!pip install pytorch-ignite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 19.9 MB/s eta 0:00:00


In [2]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
import ignite.metrics
import ignite.contrib.handlers

/usr/local/lib/python3.10/dist-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [3]:
import torchvision.transforms as transforms
from torchvision.datasets import OxfordIIITPet
from torch.utils.data.dataloader import default_collate
from torch.utils.data import DataLoader, random_split, ConcatDataset

In [4]:
class ConvNorm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=2, padding=1):
        super(ConvNorm, self).__init__()
        self.linear = nn.Conv2d(
            in_channels, out_channels, kernel_size=kernel_size,
            stride=stride, padding=padding, bias=False
        )
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.linear(x)
        x = self.bn(x)
        return x

In [5]:
class Stem16(nn.Module):
    def __init__(self):
        super(Stem16, self).__init__()
        self.conv1 = ConvNorm(3, 32)
        self.act1 = nn.Hardswish()
        self.conv2 = ConvNorm(32, 64)
        self.act2 = nn.Hardswish()
        self.conv3 = ConvNorm(64, 128)
        self.act3 = nn.Hardswish()
        self.conv4 = ConvNorm(128, 256)

    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.act2(self.conv2(x))
        x = self.act3(self.conv3(x))
        x = self.conv4(x)
        return x

In [6]:
class LinearNorm(nn.Module):
    def __init__(self, in_features, out_features):
        super(LinearNorm, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias=False)
        self.bn = nn.BatchNorm1d(out_features)

    def forward(self, x):
        # Flatten if the input is 3D to apply BatchNorm1d
        if x.dim() == 3:
            B, N, C = x.shape
            x = x.reshape(B * N, C)  # view에서 reshape로 변경
            x = self.bn(self.linear(x))
            x = x.reshape(B, N, -1)  # 원래 모양으로 다시 reshape
        else:
            x = self.bn(self.linear(x))
        return x

In [7]:
class Attention(nn.Module):
    def __init__(self, dim, num_heads, attn_ratio=2):
        super(Attention, self).__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5
        inner_dim = head_dim * num_heads * 3
        self.qkv = LinearNorm(dim, inner_dim)

        self.attention_bias = None

        self.proj = nn.Sequential(
            nn.Hardswish(),
            LinearNorm(dim, dim)
        )

    def forward(self, x):
        B, N, C = x.shape

        if self.attention_bias is None or self.attention_bias.size(1) != N:
            self.attention_bias = torch.nn.Parameter(
                torch.zeros(self.num_heads, N, N)
            )

        qkv = self.qkv(x)
        # qkv: (B, N, 3 * num_heads, head_dim) -> (3, B, num_heads, N, head_dim)
        qkv = qkv.view(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        # q = k = v: (B, num_heads, N, head_dim)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # attn: (B, num_heads, N, N)
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn += self.attention_bias.to(x.device)

        attn = attn.softmax(dim=-1)
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        return self.proj(x)

In [8]:
class LevitMlp(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(LevitMlp, self).__init__()
        self.ln1 = LinearNorm(in_features, hidden_features)
        self.act = nn.Hardswish()
        self.drop = nn.Dropout(p=0.0, inplace=False)
        self.ln2 = LinearNorm(hidden_features, out_features)

    def forward(self, x):
        x = self.ln1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.ln2(x)
        return x

In [9]:
class LevitBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=2):
        super(LevitBlock, self).__init__()
        self.attn = Attention(dim, num_heads)
        self.drop_path1 = nn.Identity()
        self.mlp = LevitMlp(dim, dim * mlp_ratio, dim)
        self.drop_path2 = nn.Identity()

    def forward(self, x):
        x = x + self.drop_path1(self.attn(x))
        x = x + self.drop_path2(self.mlp(x))
        return x

In [10]:
class AttentionDownsample(nn.Module):
    def __init__(self, dim, out_dim, num_heads, attn_ratio=2):
        super(AttentionDownsample, self).__init__()
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        inner_dim = dim * attn_ratio * num_heads
        self.kv = LinearNorm(dim, inner_dim)

        # Downsample 적용
        self.q = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=2, stride=2),
            nn.Flatten(start_dim=1)
        )

        # proj 부분 수정
        self.proj = nn.Sequential(
            nn.Hardswish(),
            LinearNorm(dim, out_dim)  # dim과 out_dim이 올바르게 설정되었는지 확인
        )

    def forward(self, x):
        B, N, C = x.shape # N = H * W
        H = W = int(N ** 0.5)  # N의 제곱근을 통해 H, W 결정 (e.g., N=196 -> H=W=14)
        x = x.reshape(B, C, H, W)  # view에서 reshape로 변경하여 연속된 메모리 레이아웃 문제 해결

        kv = self.kv(x.flatten(2).transpose(1, 2))  # kv 생성
        q = self.q(x)  # 다운샘플링된 q 생성

        # 다운샘플링 후 적절한 차원으로 변환
        q = q.reshape(B, -1, C)  # view에서 reshape로 변경
        x = self.proj(q)
        return x

In [11]:
class LevitDownsample(nn.Module):
    def __init__(self, dim, out_dim, num_heads, attn_ratio=2):
        super(LevitDownsample, self).__init__()
        self.attn_downsample = AttentionDownsample(dim, out_dim, num_heads, attn_ratio)
        self.mlp = LevitMlp(out_dim, out_dim * attn_ratio, out_dim)
        self.drop_path = nn.Identity()

    def forward(self, x):
        x = self.attn_downsample(x)
        x = self.drop_path(self.mlp(x))
        return x

In [12]:
class LevitStage(nn.Module):
    def __init__(self, dim, out_dim, num_heads, num_blocks, downsample=True):
        super(LevitStage, self).__init__()
        self.downsample = LevitDownsample(dim, out_dim, num_heads) if downsample else nn.Identity()
        self.blocks = nn.Sequential(*[LevitBlock(out_dim, num_heads) for _ in range(num_blocks)])

    def forward(self, x):
        x = self.downsample(x)
        x = self.blocks(x)
        return x

In [13]:
class NormLinear(nn.Module):
    def __init__(self, in_features, out_features, dropout_prob=0.0):
        super(NormLinear, self).__init__()
        self.bn = nn.BatchNorm1d(in_features)
        self.drop = nn.Dropout(p=dropout_prob, inplace=False)
        self.linear = nn.Linear(in_features, out_features, bias=True)

    def forward(self, x):
        x = self.bn(x)
        x = self.drop(x)
        x = self.linear(x)
        return x

In [14]:
class Levit(nn.Module):
    def __init__(self, num_classes = 37):
        super(Levit, self).__init__()

        self.stem = Stem16()

        self.stages = nn.Sequential(
            LevitStage(256, 256, 4, 4, downsample=False),
            LevitStage(256, 384, 6, 4, downsample=True),
            LevitStage(384, 512, 8, 4, downsample=True),
        )

        self.head = NormLinear(512, num_classes)

    def forward(self, x):
        x = self.stem(x)

        B, C, H, W = x.shape
        x = x.view(B, C, H * W).transpose(1, 2)
        x = self.stages(x)

        x = x.mean(dim=1)
        out = self.head(x)

        return out


In [15]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [16]:
trainval_data = datasets.OxfordIIITPet(root="data", split="trainval", target_types="category", download=True, transform=train_transform)
test_data = datasets.OxfordIIITPet(root="data", split="test", target_types="category", download=True, transform=train_transform)
combined_data = ConcatDataset([trainval_data, test_data])

train_size = int(0.7 * len(combined_data))
val_size = int(0.15 * len(combined_data))
test_size = len(combined_data) - train_size - val_size
train_data, val_data, test_data = random_split(combined_data, [train_size, val_size, test_size])

100%|██████████| 792M/792M [00:37<00:00, 21.3MB/s]


Extracting data/oxford-iiit-pet/images.tar.gz to data/oxford-iiit-pet


100%|██████████| 19.2M/19.2M [00:02<00:00, 9.11MB/s]


Extracting data/oxford-iiit-pet/annotations.tar.gz to data/oxford-iiit-pet


In [17]:
model = Levit()
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

batch_size = 32
learning_rate = 0.001
num_epochs = 50

Levit(
  (stem): Stem16(
    (conv1): ConvNorm(
      (linear): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act1): Hardswish()
    (conv2): ConvNorm(
      (linear): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act2): Hardswish()
    (conv3): ConvNorm(
      (linear): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act3): Hardswish()
    (conv4): ConvNorm(
      (linear): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (stages): Sequential(
    (0): Le

In [18]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"Train set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")

Train set size: 5144
Validation set size: 1102
Test set size: 1103


In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
from tqdm import tqdm
import time

In [21]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {accuracy:.2f}%")

In [22]:
def evaluate(model, data_loader, criterion, device, phase="Validation"):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc=f"{phase}"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(data_loader)
    accuracy = 100 * correct / total
    print(f"{phase} Loss: {epoch_loss:.4f}, {phase} Accuracy: {accuracy:.2f}%")

In [23]:
def measure_inference_time(model, data_loader, device):
    model.eval()
    times = []

    with torch.no_grad():
        for inputs, _ in data_loader:
            inputs = inputs.to(device)
            start_time = torch.cuda.Event(enable_timing=True)
            end_time = torch.cuda.Event(enable_timing=True)

            start_time.record()
            _ = model(inputs)  # inference 수행
            end_time.record()

            # 시간 측정
            torch.cuda.synchronize()  # CUDA에서 모든 커널이 완료될 때까지 대기
            elapsed_time = start_time.elapsed_time(end_time)  # 밀리초 단위로 반환
            times.append(elapsed_time)

    # 통계량 계산
    times_np = np.array(times)
    total_inferences = len(times_np)
    avg_time = np.mean(times_np)
    std_dev = np.std(times_np)
    max_time = np.max(times_np)
    min_time = np.min(times_np)

    # 결과 출력
    print(f"Inference Time Measurement Results:")
    print(f"Total Inferences: {total_inferences}")
    print(f"Average Time: {avg_time:.2f} ms")
    print(f"Standard Deviation: {std_dev:.2f} ms")
    print(f"Maximum Time: {max_time:.2f} ms")
    print(f"Minimum Time: {min_time:.2f} ms")

    return times

In [24]:
def inference_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = "{:.2f}".format(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
total_time = 0
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train(model, train_loader, criterion, optimizer, device)
    start_time = time.monotonic()
    evaluate(model, val_loader, criterion, device, phase="Validation")
    end_time = time.monotonic()

    inference_min, inference_sec = inference_time(start_time, end_time)
    print(f"Validation Inference Time: {inference_min}m {inference_sec}s")
    total_time += end_time - start_time


Epoch 1/50


Training: 100%|██████████| 161/161 [00:34<00:00,  4.73it/s]


Train Loss: 3.7573, Train Accuracy: 2.35%


Validation: 100%|██████████| 35/35 [00:05<00:00,  6.02it/s]


Validation Loss: 3.8829, Validation Accuracy: 3.27%
Validation Inference Time: 0m 5.83s

Epoch 2/50


...

Validation Loss: 6.7203, Validation Accuracy: 11.62%
Validation Inference Time: 0m 5.60s

Epoch 50/50


Training: 100%|██████████| 161/161 [00:31<00:00,  5.11it/s]


Train Loss: 0.1482, Train Accuracy: 95.59%


Validation: 100%|██████████| 35/35 [00:05<00:00,  6.23it/s]

Validation Loss: 6.9448, Validation Accuracy: 11.80%
Validation Inference Time: 0m 5.62s


In [26]:
print(f"\nTotal Validation inference Time: {int(total_time / 60)}m {int(total_time % 60)}s")


Total Validation inference Time: 4m 41s


In [27]:
start_time = time.monotonic()
evaluate(model, test_loader, criterion, device, phase="Test")
end_time = time.monotonic()
inference_min, inference_sec = inference_time(start_time, end_time)
print(f"Test Inference Time: {inference_min}m {inference_sec}s")

Test: 100%|██████████| 35/35 [00:05<00:00,  6.08it/s]

Test Loss: 6.7993, Test Accuracy: 12.96%
Test Inference Time: 0m 5.76s


In [28]:
measure_inference_time(model, test_loader, device)

Inference Time Measurement Results:
Total Inferences: 35
Average Time: 16.29 ms
Standard Deviation: 0.52 ms
Maximum Time: 18.00 ms
Minimum Time: 15.14 ms


[17.75040054321289,
 16.297632217407227,
 16.34662437438965,
 16.376544952392578,
 16.151775360107422,
 16.257568359375,
 17.593408584594727,
 18.001535415649414,
 16.577024459838867,
 16.183168411254883,
 16.274784088134766,
 16.169984817504883,
 16.20684814453125,
 16.212352752685547,
 16.783327102661133,
 16.384735107421875,
 16.300800323486328,
 16.253599166870117,
 15.899040222167969,
 15.981280326843262,
 15.967519760131836,
 15.992351531982422,
 15.847264289855957,
 16.262592315673828,
 16.223648071289062,
 16.058015823364258,
 16.05615997314453,
 15.964672088623047,
 15.991423606872559,
 16.2508487701416,
 16.02617645263672,
 16.22211265563965,
 16.09075164794922,
 16.119136810302734,
 15.14243221282959]